<a href="https://colab.research.google.com/github/Manas1820/WannaCry_Ransomware/blob/main/WannaCry_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras import regularizers
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint
import numpy as np
import tensorflow as tf
import sys
from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc,roc_curve, recall_score, classification_report, f1_score,precision_recall_fscore_support)

In [7]:
# Retrieve and prepare data
wannacry_data = pd.read_csv("https://raw.githubusercontent.com/Manas1820/WannaCry_Ransomware/main/Dataset/wannacry.csv")
benign_data = pd.read_csv("https://raw.githubusercontent.com/Manas1820/WannaCry_Ransomware/main/Dataset/malacious_data.csv").sample(frac = 1)
malware_data = pd.read_csv("https://raw.githubusercontent.com/Manas1820/WannaCry_Ransomware/main/Dataset/malware_data.csv").sample(frac = 1)
malware_train, malware_test = train_test_split(malware_data, test_size = 0.08)

In [8]:
# Autoencoder design
input_dim = malware_train.shape[1]
input_layer = Input(shape=(input_dim, ))
nodes_number = 17
while (nodes_number>=3):
    if nodes_number == 17:
        layer = Dense(nodes_number, activation='tanh')(input_layer)
    else:
        layer = Dense(nodes_number, activation='tanh')(layer)
    nodes_number=nodes_number-2
nodes_number=5
while(nodes_number<=17):
    layer = Dense(nodes_number, activation='tanh')(layer)
    nodes_number=nodes_number+2
output_layer = Dense(input_dim, activation='tanh')(layer)
autoencoder = Model(inputs=input_layer, outputs=output_layer)

nb_epoch = 10
batch_size = 18
autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath="autoencoder.h5", verbose=0, save_best_only=True)

In [5]:
history = autoencoder.fit(malware_train, malware_train, epochs=nb_epoch, batch_size=batch_size, shuffle=True, validation_data=(malware_test, malware_test), verbose=1, callbacks=[checkpointer]).history
autoencoder = load_model('autoencoder.h5')

predictions_malware = autoencoder.predict(malware_test)
mse_malware = np.mean(np.power(malware_test - predictions_malware, 2), axis=1)

predictions_benign_data = autoencoder.predict(benign_data)
mse_benign = np.mean(np.power(benign_data - predictions_benign_data, 2), axis=1)

predictions_wannacry_data = autoencoder.predict(wannacry_data)
mse_wannacry_data = np.mean(np.power(wannacry_data - predictions_wannacry_data, 2), axis=1)

threshold = np.average(mse_malware)

Epoch 1/20
310/310 [==============================] - 2s 5ms/step - loss: 11462674.0000 - accuracy: 0.5363 - val_loss: 12023563.0000 - val_accuracy: 0.5402
Epoch 2/20
310/310 [==============================] - 3s 9ms/step - loss: 11462677.0000 - accuracy: 0.5363 - val_loss: 12023563.0000 - val_accuracy: 0.5402
Epoch 3/20
310/310 [==============================] - 2s 7ms/step - loss: 11462674.0000 - accuracy: 0.5363 - val_loss: 12023563.0000 - val_accuracy: 0.5402
Epoch 4/20
310/310 [==============================] - 2s 5ms/step - loss: 11462674.0000 - accuracy: 0.5363 - val_loss: 12023563.0000 - val_accuracy: 0.5402
Epoch 5/20
310/310 [==============================] - 2s 6ms/step - loss: 11462680.0000 - accuracy: 0.5363 - val_loss: 12023563.0000 - val_accuracy: 0.5402
Epoch 6/20
310/310 [==============================] - 2s 5ms/step - loss: 11462679.0000 - accuracy: 0.5363 - val_loss: 12023563.0000 - val_accuracy: 0.5402
Epoch 7/20
310/310 [==============================] - 2s 6ms/ste

In [9]:
acc=0
rec = 0

# fine tune the threshold if needed
while rec<.6 or acc<.6 :
    threshold = threshold - 0.0001
    print(threshold)
    TP,FP,TN,FN=0,0,0,0
    for e in mse_malware:
        if e<threshold:
            TP+=1
        else:
            FN+=1
    for e in mse_benign:
        if e>=threshold:
            TN+=1
        else:
            FP+=1
    try:
        acc = 1.*(TP+TN)/(TP+TN+FP+FN)
        rec = 1.*(TP)/(TP+FN)
        prec = 1. * TP / (TP + FP)
        print (f"TP:{TP}\nFP:{FP}\nTN:{TN}\nFN:{FN}")
        print (f"Accuracy: {acc}\nRecall: {rec}\nPrecision: {prec}")
    except:
        pass

12023563.338388078
TP:439
FP:234
TN:138
FN:46
Accuracy: 0.6732788798133023
Recall: 0.9051546391752577
Precision: 0.6523031203566122
